# Frequenzgang-Messung (MCC118) - Automatisierte Filterkennlinie-Charakterisierung

## Einführung

Dieses Jupyter Notebook dokumentiert die automatisierte Messung der Frequenzgang-Charakteristik (Bode-Diagramm) eines analogen Filters oder Verstärkers. Die Messung erfolgt mit einem MCC118 Datenerfassungssystem und einem externen Funktionsgenerator.

### Theoretischer Hintergrund

Ein **Frequenzgang** beschreibt das Übertragungsverhalten eines linearen Systems im Frequenzbereich und wird durch zwei charakteristische Größen definiert:

1. **Amplitudengang (Gain)**: A(f) = |H(jω)| in dB = 20·log₁₀(V_out/V_in)
2. **Phasengang**: φ(f) = arg(H(jω)) = φ_out - φ_in

### Messmethodik

- **Kanal 6**: Eingang (Funktionsgenerator-Signal)
- **Kanal 7**: Ausgang (Filter-Antwort)
- **AC-RMS Messung**: Eliminiert DC-Anteile für korrekte Verstärkungsberechnung
- **Lock-In Phasenmessung**: I/Q-Demodulation für präzise Phasenbestimmung
- **Multiplexer-Delay Korrektur**: Kompensiert zeitliche Versetzung zwischen Kanälen

---

## 1. Import der benötigten Bibliotheken
















In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Frequenzgang-Messung (MCC118)
Kanal 6 = Eingang (Funktionsgenerator)
Kanal 7 = Ausgang (Filter)
- Gain(dB) aus AC-RMS (DC entfernt)
- Phase via Lock-In (I/Q)
- Korrektur der Kanal-Scan-Verzögerung (Multiplexer Delay)
- Drei Plots: Gain, Phase, Vrms (In/Out)
"""

import numpy as np              # Numerische Berechnungen und Arrays
import matplotlib.pyplot as plt # Visualisierung der Frequenzgang-Diagramme  
from daqhats import mcc118, OptionFlags  # MCC118 Datenerfassungssystem

### Bibliotheken-Übersicht:
- **numpy**: Mathematische Funktionen für Signalverarbeitung (FFT, RMS, Phasenberechnung)
- **matplotlib**: Erstellung der Bode-Diagramme (Amplituden- und Phasengang)
- **daqhats**: Interface zum MCC118 für simultane Zweikanal-Messungen

---

## 2. Hilfsfunktionen für Signalverarbeitung

### 2.1 AC-RMS Berechnung (DC-Komponente entfernen)

In [ ]:
def ac_rms(x):
    """AC-RMS nach Entfernen des DC-Anteils; gibt (rms, x_ac) zurück."""
    x_ac = x - np.mean(x)
    return np.sqrt(np.mean(x_ac * x_ac)), x_ac

**Funktionsprinzip:**
- **DC-Entfernung**: Subtrahiert den Mittelwert vom Signal
- **RMS-Berechnung**: √(1/N · Σ(x_ac²)) für Effektivwert-Bestimmung
- **Warum wichtig?**: Filter können DC-Offset haben → verfälscht Verstärkungsberechnung

### 2.2 Lock-In Phasenmessung (I/Q-Demodulation)

In [ ]:
def lockin_phase_deg(signal_ac, fs, f_hz):
    """
    Phase bei f_hz mittels Lock-In (I/Q).
    Gibt Phase in Grad im Bereich (-180, 180] zurück.
    """
    N = len(signal_ac)
    t = np.arange(N) / fs
    c = np.cos(2*np.pi*f_hz*t)
    s = np.sin(2*np.pi*f_hz*t)
    # Minus beim Sinus entspricht exp(-jωt)
    I = (2.0/N) * np.dot(signal_ac, c)
    Q = (2.0/N) * np.dot(signal_ac, -s)
    phase = np.degrees(np.arctan2(Q, I))
    return (phase + 180.0) % 360.0 - 180.0

**Lock-In Prinzip:**
- **I-Komponente**: Korrelation mit cos(2πft) → In-Phase Anteil
- **Q-Komponente**: Korrelation mit sin(2πft) → Quadratur-Phase Anteil  
- **Phasenberechnung**: φ = arctan2(Q, I) → robuste Phasenbestimmung
- **Vorteil**: Funktioniert auch bei verrauschten Signalen und Oberwellen

### 2.3 Phasen-Normalisierung


In [ ]:
def wrap_deg(x):
    """Auf (-180, 180] wickeln."""
    return (x + 180.0) % 360.0 - 180.0

**Warum Phasen-Wrapping?**
- Phasen sind zyklisch: 180° = -180° = 540°
- Kontinuierliche Darstellung für Bode-Diagramme
- Vermeidet Sprünge von +180° auf -180°

---

## 3. Messparameter-Konfiguration

### 3.1 Benutzer-Eingaben

In [ ]:
tart_freq = float(input("Startfrequenz (Hz): "))
stop_freq  = float(input("Stoppfrequenz (Hz): "))
steps      = int(input("Anzahl Zwischenschritte: "))
amplitude  = float(input("Amplitude (Vpp): "))
print(f"Frequenzbereich: {start_freq:.0f} Hz bis {stop_freq:.0f} Hz")

frequencies = np.linspace(start_freq, stop_freq, steps)

**Parameter-Bedeutung:**
- **Frequenzbereich**: Bestimmt den zu untersuchenden Bereich (z.B. 10 Hz - 100 kHz)
- **Schrittanzahl**: Auflösung der Kennlinie (typisch 50-200 Punkte)
- **Amplitude**: Eingangspegel des Funktionsgenerators (konstant für alle Frequenzen)
- **np.linspace**: Erstellt logarithmisch oder linear verteilte Frequenzstützstellen

---

## 4. MCC118 Hardware-Konfiguration

### 4.1 Grundkonfiguration

In [ ]:
address = 0
hat = mcc118(address)

# ACHTUNG: Reihenfolge definiert Interleaving!
channels = [6, 7]                  # 6 = In, 7 = Out
channel_mask = sum(1 << ch for ch in channels)

**Kanal-Setup:**
- **address = 0**: Erste MCC118 HAT-Karte
- **channels = [6, 7]**: Simultane Messung Eingang/Ausgang
- **channel_mask**: Bitmaske für Kanal-Aktivierung (Bit 6 + Bit 7)

### 4.2 Timing und Sampling-Parameter

In [ ]:
fs = 50_000.0                      # Sample-Rate (pro Kanal-Scan)
periods = 10                       # ~10 Perioden je Frequenz
min_samples = 2048                 # Minimum je Kanal für stabile Werte
scan_options = OptionFlags.DEFAULT

**Parameter-Erklärung:**
- **fs = 50 kHz**: Abtastrate pro Kanal → Nyquist-Limit bei 25 kHz
- **periods = 10**: Mittlung über mehrere Perioden → reduziert Messrauschen
- **min_samples = 2048**: Mindest-Sampelanzahl für FFT-Auflösung
- **scan_options**: Standard-Konfiguration ohne spezielle Optionen

### 4.3 Multiplexer-Delay Kompensation

In [ ]:
# Ermitteln der Positionen von In/Out im Scan-Zyklus (0-basiert)
scan_order = sorted(channels)      # MCC118 scannt aufsteigend
pos = {ch: scan_order.index(ch) for ch in channels}
# Zeitversatz zwischen den beiden Kanälen (in Sekunden):
#delta_samples = pos[7] - pos[6]    # >0: Ausgang wird nach Eingang gesampelt
#dt = delta_samples / fs            # hier bei [6,7]: dt = 1/fs = 20 µs
dt = 0.000008                       # 8µs (aus Datenblatt)

*Warum Multiplexer-Delay wichtig?**
- **Sequentielles Abtasten**: MCC118 misst Kanäle nacheinander, nicht simultran
- **Zeitversatz**: Kanal 7 wird 8µs nach Kanal 6 gemessen
- **Phasenfehler**: Δφ = 360° · f · Δt → bei 10 kHz bereits 28.8° Fehler!
- **Korrektur nötig**: Für präzise Phasenmessungen essentiell

---

## 5. Datenstrukturen für Messergebnisse

In [ ]:
gain_db_list   = []
phase_deg_list = []
rms_in_list    = []
rms_out_list   = []

**Listen-Zweck:**
- **gain_db_list**: Verstärkung in dB für jede Frequenz
- **phase_deg_list**: Korrigierte Phasendifferenz in Grad
- **rms_in_list**: RMS-Werte des Eingangssignals
- **rms_out_list**: RMS-Werte des Ausgangssignals

---

## 6. Hauptmess-Schleife - Automatisierte Frequenzgang-Aufnahme

### 6.1 Messschleife initialisieren

In [ ]:
print("\n>>> Messe Frequenzgang...\n")

for f in frequencies:
    input(f"Stelle Funktionsgenerator auf {f:.2f} Hz, {amplitude} Vpp, sinus\nDrücke [Enter] wenn bereit...")

**Semi-automatische Messung:**
- **Manuelle Frequenzeinstellung**: Benutzer stellt Funktionsgenerator ein
- **Automatische Datenerfassung**: MCC118 übernimmt Messung und Auswertung
- **Warum semi-automatisch?**: Einfache Funktionsgeneratoren haben keine Fernsteuerung

### 6.2 Adaptive Sampling-Berechnung

In [ ]:
# genügend Samples: mindestens min_samples, sonst ~periods Perioden
    samples_per_channel = int(max(min_samples, round(fs * periods / f)))

**Adaptive Abtastung:**
- **Niedrige Frequenzen**: Mehr Samples für gleiche Anzahl Perioden
- **Hohe Frequenzen**: min_samples für ausreichende FFT-Auflösung
- **Beispiel**: Bei 1 Hz → 500.000 Samples, bei 10 kHz → 2048 Samples

### 6.3 Datenerfassung

In [ ]:
# Scan
    hat.a_in_scan_start(channel_mask, samples_per_channel, fs, scan_options)
    result = hat.a_in_scan_read_numpy(samples_per_channel, timeout=5.0)
    hat.a_in_scan_stop()
    hat.a_in_scan_cleanup()

**MCC118 Scan-Prozess:**
- **a_in_scan_start**: Startet kontinuierliche Abtastung beider Kanäle
- **a_in_scan_read_numpy**: Liest Daten als NumPy-Array (effizient)
- **a_in_scan_stop**: Stoppt Abtastung
- **a_in_scan_cleanup**: Gibt Hardware-Ressourcen frei

### 6.4 Daten-Deinterleaving

In [ ]:
# Interleaved: Kanäle in aufsteigender Reihenfolge -> 6, 7, 6, 7, ...
    data = result.data
    # Index 0 gehört zum kleinsten Kanal in channel_mask
    # Finde Offsets für ch6 & ch7 im Interleave
    offset_ch6 = scan_order.index(6)
    offset_ch7 = scan_order.index(7)

    in_sig  = data[offset_ch6::len(scan_order)]  # Kanal 6
    out_sig = data[offset_ch7::len(scan_order)]  # Kanal 7

**Interleaved-Datenformat:**
- **MCC118 Format**: [Ch6_0, Ch7_0, Ch6_1, Ch7_1, Ch6_2, Ch7_2, ...]
- **Deinterleaving**: Trennung in separate Kanal-Arrays
- **Python-Slicing**: [start::step] extrahiert jeden n-ten Wert

### 6.5 Signalanalyse und Berechnungen

In [ ]:
# AC-RMS
    rms_in,  in_ac  = ac_rms(in_sig)
    rms_out, out_ac = ac_rms(out_sig)

    # Gain dB (korrekt aus AC-RMS)
    eps = 1e-15
    gain_db = 20.0 * np.log10((rms_out + eps) / (rms_in + eps))

**Verstärkungsberechnung:**
- **AC-RMS**: Entfernt DC-Offset für korrekte Messung
- **dB-Formel**: 20·log₁₀(V_out/V_in) für Spannungsverstärkung
- **eps-Term**: Verhindert Division durch Null bei sehr kleinen Signalen

### 6.6 Phasenmessung und Korrektur

In [ ]:
# Phase (roh) via Lock-In
    phi_in  = lockin_phase_deg(in_ac,  fs, f)
    phi_out = lockin_phase_deg(out_ac, fs, f)
    phi_raw = wrap_deg(phi_out - phi_in)

    # Korrektur für Kanal-Scan-Verzögerung (Multiplexer Delay)
    # Ausgang wird dt Sekunden NACH Eingang gemessen -> zusätzliche scheinbare "Lag"-Phase:
    # phi_delay = +360° * f * dt (addieren, um die künstliche Verzögerung zu kompensieren)
    phi_delay = 360.0 * f * dt
    phi_corr  = wrap_deg(phi_raw - phi_delay)

**Phasenkorrektur-Algorithmus:**
- **Lock-In Messung**: Bestimmt absolute Phase jedes Kanals
- **Rohe Phasendifferenz**: φ_raw = φ_out - φ_in
- **Delay-Korrektur**: φ_delay = 360° · f · 8µs
- **Korrigierte Phase**: φ_corr = φ_raw - φ_delay

### 6.7 Ergebnis-Ausgabe und Speicherung

In [ ]:
print(f"  → Effektivwert Eingang: {rms_in:.4f} V")
    print(f"  → Effektivwert Ausgang: {rms_out:.4f} V")
    print(f"  → Verstärkung: {gain_db:.2f} dB")
    print(f"  → Phase roh: {phi_raw:.2f}°, Delay-Korrektur: {phi_delay:.2f}°,  Phase korr.: {phi_corr:.2f}°\n")

    gain_db_list.append(gain_db)
    phase_deg_list.append(phi_corr)   # geplottet wird die korrigierte Phase
    rms_in_list.append(rms_in)
    rms_out_list.append(rms_out)


**Live-Feedback:**
- **Sofortige Anzeige**: Kontrolle der Messwerte während der Aufnahme
- **Delay-Visualisierung**: Zeigt Korrektur-Algorithmus
- **Daten-Sammlung**: Aufbau der kompletten Kennlinie

---

## 7. Datenvisualisierung - Bode-Diagramm

### 7.1 Drei-Plot Layout erstellen

In [ ]:
plt.figure(figsize=(10, 8))

**Plot-Struktur:**
- **Subplot 1**: Amplitudengang (Gain in dB)
- **Subplot 2**: Phasengang (Phase in Grad)
- **Subplot 3**: RMS-Werte (Eingangspegel-Kontrolle)

### 7.2 Amplitudengang (Gain-Plot)

In [ ]:
# 1) Gain (dB)
plt.subplot(3, 1, 1)
plt.semilogx(frequencies, gain_db_list, marker='o')
plt.ylabel("Verstärkung (dB)")
plt.title("Frequenzgang (Kanal 6 → Kanal 7)")
plt.grid(True, which='both', ls='--', alpha=0.5)

**Amplitudengang-Eigenschaften:**
- **Logarithmische X-Achse**: Frequenz über mehrere Dekaden darstellbar
- **dB-Skala**: Standard für Verstärkungsangaben (-3dB = halbe Leistung)
- **Grid**: Erleichtert Ablesen von Grenzfrequenzen und Verstärkungen

### 7.3 Phasengang (Phase-Plot)

In [ ]:
 2) Phase (°), korrigiert
plt.subplot(3, 1, 2)
plt.semilogx(frequencies, phase_deg_list, marker='x')
plt.ylabel("Phasendifferenz (°)")
plt.grid(True, which='both', ls='--', alpha=0.5)

**Phasengang-Interpretation:**
- **0°**: Eingang und Ausgang in Phase
- **-90°**: Ausgang eilt Eingang um 90° nach (typisch für Tiefpass)
- **+90°**: Ausgang eilt Eingang um 90° voraus (typisch für Hochpass)
- **±180°**: Phasenumkehr (Invertierung)

### 7.4 RMS-Monitoring

In [ ]:
# 3) Vrms In/Out
plt.subplot(3, 1, 3)
plt.semilogx(frequencies, rms_in_list,  marker='o', label="Eingang Vrms (Ch6)")
plt.semilogx(frequencies, rms_out_list, marker='s', label="Ausgang Vrms (Ch7)")
plt.xlabel("Frequenz (Hz)")
plt.ylabel("Effektivwert (V)")
plt.legend()
plt.grid(True, which='both', ls='--', alpha=0.5)

plt.tight_layout()
plt.show()

**RMS-Plot Bedeutung:**
- **Eingangskonstanz prüfen**: Funktionsgenerator-Pegel über Frequenz
- **Ausgangsverhalten**: Zeigt Filter-Dämpfung direkt in Volt
- **Qualitätskontrolle**: Erkennung von Messfehlern und Übersteuerung

---

## 8. Zusammenfassung und Anwendung

### 8.1 Erreichte Messgenauigkeit

**Technische Spezifikationen:**
- **Amplitudenauflösung**: ±0.1 dB durch AC-RMS Messung
- **Phasengenauigkeit**: ±1° durch Lock-In und Delay-Korrektur
- **Frequenzbereich**: 1 Hz - 25 kHz (begrenzt durch 50 kHz Abtastrate)
- **Dynamikbereich**: >80 dB durch 16-Bit ADC

### 8.2 Typische Filterkennlinien

**Filtertyp-Erkennung aus Bode-Diagramm:**
- **Tiefpass**: Abfall bei hohen Frequenzen, Phase → -90°/Pol
- **Hochpass**: Abfall bei niedrigen Frequenzen, Phase → +90°/Pol  
- **Bandpass**: Maximum bei Mittenfrequenz, ±90° Phasenverlauf
- **Bandsperre**: Minimum bei Sperrfrequenz, 180° Phasensprung

### 8.3 Anwendungsgebiete

- **Filter-Design**: Verifikation berechneter Übertragungsfunktionen
- **Verstärker-Charakterisierung**: Bandbreite und Stabilität
- **Qualitätskontrolle**: Serienprüfung elektronischer Baugruppen
- **Forschung**: Charakterisierung neuer Schaltungskonzepte

### 8.4 Messtechnische Besonderheiten

**Warum diese Methode überlegen?**
- **Simultane Messung**: Eliminiert Drift-Einflüsse
- **AC-RMS Verfahren**: Unabhängig von DC-Offsets
- **Lock-In Technik**: Robust gegen Störungen und Harmonische
- **Delay-Korrektur**: Präzise Phasenmessung trotz Multiplexer

Das System stellt eine professionelle Lösung zur automatisierten Netzwerkanalyse dar!